In [1]:
from bs4 import BeautifulSoup 
import pandas as pd
import requests 
import re

pd.set_option ('display.max_columns', None)

In [2]:
url = "https://moviesdatabase.p.rapidapi.com/titles"

headers = {
    "X-RapidAPI-Key": "431e779057msh1f0c87c2937a329p125efcjsn45064e4b4925",
    "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
}

querystring = {
    "genre": "Action",
    "startYear": "2010",
    "titleType": "movie",
    "endYear": "2024"
}

def llamar_pelis(url, headers, querystring):
    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error en la petición tipo:", response.status_code, response.reason)
        return None


In [3]:
llamar_pelis (url, headers, querystring)

{'page': 1,
 'next': '/titles?titleType=movie&endYear=2024&startYear=2010&genre=Action&page=2',
 'entries': 10,
 'results': [{'_id': '61e58c7ed8f3c0931e43e919',
   'id': 'tt10225380',
   'primaryImage': {'id': 'rm3016621313',
    'width': 1023,
    'height': 576,
    'url': 'https://m.media-amazon.com/images/M/MV5BYzBiNDdhMDAtNTliMC00NDcyLWE3MDYtZTZhNzFmMzhiOTZiXkEyXkFqcGdeQXVyODgwNTU5Nzg@._V1_.jpg',
    'caption': {'plainText': 'Multiverso (2024)', '__typename': 'Markdown'},
    '__typename': 'Image'},
   'titleType': {'displayableProperty': {'value': {'plainText': '',
      '__typename': 'Markdown'},
     '__typename': 'DisplayableTitleTypeProperty'},
    'text': 'Movie',
    'id': 'movie',
    'isSeries': False,
    'isEpisode': False,
    'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}],
    'canHaveEpisodes': False,
    '__typename': 'TitleType'},
   'titleText': {'text': 'Multiverso', '__typename': 'TitleText'},
   'originalTitleText': {'text': 'Multiverso', 

In [4]:
resultados_iniciales = llamar_pelis(url, headers, querystring)
if resultados_iniciales:
    print(resultados_iniciales)  
else:
    print("No se recibieron datos.")

{'page': 1, 'next': '/titles?titleType=movie&endYear=2024&startYear=2010&genre=Action&page=2', 'entries': 10, 'results': [{'_id': '61e58c7ed8f3c0931e43e919', 'id': 'tt10225380', 'primaryImage': {'id': 'rm3016621313', 'width': 1023, 'height': 576, 'url': 'https://m.media-amazon.com/images/M/MV5BYzBiNDdhMDAtNTliMC00NDcyLWE3MDYtZTZhNzFmMzhiOTZiXkEyXkFqcGdeQXVyODgwNTU5Nzg@._V1_.jpg', 'caption': {'plainText': 'Multiverso (2024)', '__typename': 'Markdown'}, '__typename': 'Image'}, 'titleType': {'displayableProperty': {'value': {'plainText': '', '__typename': 'Markdown'}, '__typename': 'DisplayableTitleTypeProperty'}, 'text': 'Movie', 'id': 'movie', 'isSeries': False, 'isEpisode': False, 'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}], 'canHaveEpisodes': False, '__typename': 'TitleType'}, 'titleText': {'text': 'Multiverso', '__typename': 'TitleText'}, 'originalTitleText': {'text': 'Multiverso', '__typename': 'TitleText'}, 'releaseYear': {'year': 2024, 'endYear': None, '_

In [5]:
def llamar_pelis(url, headers, querystring):
    datos_paginas = []
    base_url = "https://moviesdatabase.p.rapidapi.com"  
    while url:
        
        if not url.startswith('http'):
            url = base_url + url
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            json_pelis = response.json()
            datos_paginas.extend(json_pelis['results'])  
            url = json_pelis.get('next')  
            querystring = {}  
        else:
            print("Error en la solicitud:", response.status_code, response.reason)
            return None
    return {"Results": datos_paginas}

In [6]:
datos_limpios = llamar_pelis(url, headers, querystring)
if datos_limpios:
    print(datos_limpios)  
else:
    print("Opsss, parece que no se han recibido datos")

Error en la solicitud: 429 Too Many Requests
Opsss, parece que no se han recibido datos


In [7]:
def limpiar_json(datos_json):
    
    peliculas = []

    
    for pelicula in datos_json['results']:
        tipo = pelicula['titleType']['text']
        nombre = pelicula['titleText']['text']
        year = pelicula['releaseYear']['year']
        mes = pelicula['releaseDate']['month'] if 'releaseDate' in pelicula and pelicula['releaseDate'] is not None else 'Desconocido'
        id_pelicula = pelicula['id']

        
        peliculas.append({
            'Tipo': tipo,
            'Nombre': nombre,
            'Año de Estreno': year,
            'Mes de Estreno': mes,
            'ID': id_pelicula
        })

    
    dataframe_peliculas = pd.DataFrame(peliculas)
    
    return dataframe_peliculas

In [8]:
datos_limpios = llamar_pelis(url, headers, querystring)
df = limpiar_json(datos_limpios)

Error en la solicitud: 429 Too Many Requests


TypeError: 'NoneType' object is not subscriptable

In [ ]:
df

In [ ]:
df.to_csv("Todas las películas de Acción.csv", index=False)  # Guarda el DataFrame en un archivo CSV sin incluir el índice.